<a href="https://colab.research.google.com/github/KIMDOKYOUNG/DeepLearning/blob/master/2_chapter2/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 밑바닥부터 시작하는 딥러닝 2 뽀개기
#### CHAPTER 2 자연어와 단어의 분산 표현
#### 일자 : 2020-08-25


- 시소러스?
  - 유의어 사전
  - 뜻이 같은 단어(동의어)와 뜻이 비슷한 단어(유의어)가 한 그룹으로 분류되어 있다.
  - 가장 유명한 시소러스 : WordNet
  - 문제점 
    - 신조어가 계속 생긴다
    - 사람을 쓴느 비용이 크다
    - 단어의 미묘한 차이를 표현할 수 없다.

- 시소러스가 가지고 있는 문제를 해결하기 위해선! 
  - 통계 기반 기법
  - 신경망을 사용한 추론 기반 기법



## 통계 기반 기법 
### corpus(말뭉치) 전처리하기


In [22]:
import numpy as np

In [4]:
text = '안녕하세요 한림대학교에 재학중인 김도경입니다. 열심히 공부하고 있어요.'
text.lower()# 영어일 경우 
text = text.replace('.',' .')
text

'안녕하세요 한림대학교에 재학중인 김도경입니다 . 열심히 공부하고 있어요 .'

In [5]:
words = text.split(' ')
words

['안녕하세요', '한림대학교에', '재학중인', '김도경입니다', '.', '열심히', '공부하고', '있어요', '.']

In [6]:
word_to_id = {}
id_to_word = {}

for word in words:
  if word not in word_to_id:
    new_id = len(word_to_id)
    word_to_id[word] = new_id
    id_to_word[new_id] = word

In [7]:
word_to_id

{'.': 4,
 '공부하고': 6,
 '김도경입니다': 3,
 '안녕하세요': 0,
 '열심히': 5,
 '있어요': 7,
 '재학중인': 2,
 '한림대학교에': 1}

In [8]:
id_to_word

{0: '안녕하세요',
 1: '한림대학교에',
 2: '재학중인',
 3: '김도경입니다',
 4: '.',
 5: '열심히',
 6: '공부하고',
 7: '있어요'}

In [16]:
[w for w in words]

['안녕하세요', '한림대학교에', '재학중인', '김도경입니다', '.', '열심히', '공부하고', '있어요', '.']

In [20]:
corpus = [word_to_id[w] for w in words]

In [23]:
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 5, 6, 7, 4])

In [24]:
def preprocess(text):
  text.lower()# 영어일 경우
  text = text.replace('.',' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}

  for word in words:
    if word not in word_to_id:
      id_idx = len(word_to_id)
      word_to_id[word] = id_idx
      id_to_word[id_idx] = word
  
  corpus = [word_to_id[w] for w in words]
  corpus = np.array(corpus)

  return corpus, word_to_id, id_to_word


In [25]:
text = '안녕하세요 한림대학교에 재학중인 김도경입니다. 열심히 공부하고 있어요.'
corpus, word_to_id, id_to_word = preprocess(text)

In [26]:
print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 5 6 7 4]
{'안녕하세요': 0, '한림대학교에': 1, '재학중인': 2, '김도경입니다': 3, '.': 4, '열심히': 5, '공부하고': 6, '있어요': 7}
{0: '안녕하세요', 1: '한림대학교에', 2: '재학중인', 3: '김도경입니다', 4: '.', 5: '열심히', 6: '공부하고', 7: '있어요'}


###  분포가설
- 분포가설이란 단어의 의미는 주변 단어에 의해 형성된다는 것이다.

### 동시발생 행렬

In [27]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'You', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'I', 5: 'hello', 6: '.'}


In [ ]:
for i in len(word_to_id):
  
